In [42]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from threading import Thread
from neurosdk.scanner import Scanner
from neurosdk.sensor import Sensor
from neurosdk.brainbit_sensor import BrainBitSensor
from neurosdk.cmn_types import *


class EEGRecorder:
    def __init__(self, sensor_family=SensorFamily.LEBrainBitFlex):
        self.scanner = Scanner([sensor_family])
        self.sensor = None
        self.data_list = []
        self.recording = False
        self.df = pd.DataFrame()
        self.running = False

    def sensor_found(self, scanner, sensors):
        print("Scanning for sensors...")
        if sensors:
            print(f"Sensor found: {sensors[0]}")
            return sensors[0]
        return None

    def connect_sensor(self):
        """ Scans and connects to the first available sensor. """
        self.scanner.sensorsChanged = self.sensor_found
        self.scanner.start()

        while not self.scanner.sensors():
            time.sleep(1)

        self.scanner.stop()

        sensors_info = self.scanner.sensors()
        if sensors_info:
            self.sensor = self.scanner.create_sensor(sensors_info[0])
            self.sensor.sensorStateChanged = self.on_sensor_state_changed
            self.sensor.batteryChanged = self.on_battery_changed
            self.sensor.signalDataReceived = self.on_signal_data_received
            print(f"Connected to sensor: {self.sensor.name}")
        else:
            raise RuntimeError("No sensor found!")

    def on_sensor_state_changed(self, sensor, state):
        print(f"Sensor {sensor.name} state: {state}")

    def on_battery_changed(self, sensor, battery):
        print(f"Battery level: {battery}%")

    def on_signal_data_received(self, sensor, data):
        """ Processes received EEG signal data. """
        if self.recording:
            timestamp = time.time()
            for package in data:
                if hasattr(package, "Samples") and isinstance(package.Samples, (list, tuple)):
                    row = [timestamp] + package.Samples
                    self.data_list.append(row)
                    self.df = pd.concat([self.df, pd.DataFrame([row], columns=["Timestamp"] + [f"Channel_{i}" for i in range(len(package.Samples))])], ignore_index=True)

    def start_recording(self):
        """ Starts EEG data recording. """
        if not self.sensor:
            raise RuntimeError("Sensor not initialized!")

        self.data_list = []
        self.df = pd.DataFrame()
        self.recording = True
        self.running = True
        self.sensor.exec_command(SensorCommand.StartSignal)
        print("Recording started...")

    def stop_recording(self):
        """ Stops EEG data recording. """
        if not self.sensor:
            raise RuntimeError("Sensor not initialized!")

        self.recording = False
        self.running = False
        self.sensor.exec_command(SensorCommand.StopSignal)
        print("Recording stopped.")

    def save_to_csv(self, filename="eeg_data.csv"):
        """ Saves recorded EEG data to a CSV file. """
        if not self.df.empty:
            self.df.to_csv(filename, index=False)
            print(f"Data saved to {filename}")
        else:
            print("No data recorded.")

    def cleanup(self):
        """ Disconnects the sensor and releases resources. """
        if self.sensor:
            self.sensor.disconnect()
            print("Sensor disconnected.")

    def record_eeg(self, duration=10, filename="eeg_data.csv"):
        """ Records EEG data for a specified duration and saves it. """
        self.start_recording()
        time.sleep(duration)
        self.stop_recording()
        self.save_to_csv(filename)
        self.cleanup()
        return self.df

if __name__ == "__main__":
    recorder = EEGRecorder()
    recorder.connect_sensor()
    recorder.record_eeg(duration=10, filename="eeg_data2.csv")


Scanning for sensors...
Sensor found: SensorInfo(SensFamily=<SensorFamily.LEBrainBitFlex: 20>, SensModel=3, Name='Flex Pro', Address='FFFC2007-2DD2-631D-889F-527FEBC96975', SerialNumber='04080029', PairingRequired=False, RSSI=-56)
Connected to sensor: Flex Pro
Battery level: 100%Recording started...

Recording stopped.
Data saved to eeg_data2.csv
Sensor disconnected.
Sensor Flex Pro state: SensorState.StateOutOfRange


In [47]:
pd.DataFrame(recorder.data_list)


,0,1,2,3,4,5,6,7,8
0,1.741177e+09,-0.014278,-0.037211,-0.014788,0.002484,0.016814,0.018359,0.012640,0.032858
1,1.741177e+09,-0.013980,-0.038597,-0.014295,0.000712,0.015734,0.018654,0.017346,0.034894
2,1.741177e+09,-0.015701,-0.039415,-0.018065,-0.000925,0.013689,0.014500,0.014200,0.032101
3,1.741177e+09,-0.016696,-0.038328,-0.020175,-0.000182,0.013515,0.012119,0.008436,0.028704
4,1.741177e+09,-0.015581,-0.036442,-0.017673,0.002100,0.015411,0.014635,0.007348,0.028855
...,...,...,...,...,...,...,...,...,...
2487,1.741177e+09,-0.010297,-0.027598,-0.010638,0.000374,0.011153,0.012705,0.012046,0.024509
2488,1.741177e+09,-0.011608,-0.028446,-0.013555,-0.000956,0.009568,0.009792,0.010014,0.022767
2489,1.741177e+09,-0.012696,-0.028054,-0.015680,-0.000733,0.009089,0.007784,0.005378,0.020188
2490,1.741177e+09,-0.012182,-0.026898,-0.014240,0.000836,0.010374,0.009337,0.003999,0.020057
